In [3]:
import $file.common
import spark._
import common._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types.{IntegerType, StringType, StructType}
import spark.implicits._
import spark.sqlContext.implicits._

import $file.$     

import spark._

import common._

import org.apache.spark.sql.functions._

import org.apache.spark.sql.types.{IntegerType, StringType, StructType}

import spark.implicits._

import spark.sqlContext.implicits._

In [4]:
import plotly._
import plotly.element._
import plotly.layout._
import plotly.Almond._

import plotly._

import plotly.element._

import plotly.layout._

import plotly.Almond._

## Consulta con DataSet

In [5]:
def infectionDS = spark.read
.option("header", "true")
.option("charset", "UTF8")
.option("delimiter",",")
.option("inferSchema", "true")
.csv("../datasets/covidworldwide.csv")
.withColumnRenamed("countriesAndTerritories","Country")
.as[(String,String,String,String,Double,Double,String,String,String,String,String,String)]

defined function infectionDS

In [6]:
def populationDS = spark.read
.option("header", "true")
.option("charset", "UTF8")
.option("delimiter",",")
.option("inferSchema", "true")
.csv("../datasets/population_by_country_2020.csv")
.withColumnRenamed("Country (or dependency)","Country")
.withColumnRenamed("Population (2020)","Population")
.as[(String,Float,String,Float,Float,Float,Double,String,String,String,String)]

defined function populationDS

### Media diaria de infecciones por número de habitante

In [7]:
def meanInfectionPerPopulationDS = 
infectionDS.join(populationDS, "Country")
        .select($"Country",
                $"dateRep" as "date",
                $"cases",
                $"Land Area (Km\u00b2)",
               $"cases" / $"Population" as "infection Per Population")
        .groupBy("country")
        .avg("infection Per Population")
        .orderBy(desc("avg(infection Per Population)"))
        .as[(String,Double)]

defined function meanInfectionPerPopulationDS

### Porcentaje diario de infectados

In [8]:
def diaryInfectionPerPopulationDS = 
infectionDS.join(populationDS, "Country")
        .select($"Country",
                $"dateRep" as "date",
                $"cases",
                $"Land Area (Km\u00b2)",
               $"cases" / $"Population" as "infection Per Population")
        .orderBy($"date")
        .as[(String,String,Int,String,Double)]

defined function diaryInfectionPerPopulationDS

## Consulta con DataFrame

In [9]:
def dfCovid = spark.read
.option("header", "true")
.option("charset", "UTF8")
.option("delimiter",",")
.option("inferSchema", "true")
.csv("../datasets/covidworldwide.csv")

defined function dfCovid

In [10]:
def dfPopulation = spark.read
.option("header", "true")
.option("charset", "UTF8")
.option("delimiter",",")
.option("inferSchema", "true")
.csv("../datasets/population_by_country_2020.csv")
.withColumnRenamed("Country (or dependency)","Country")
.withColumnRenamed("Population (2020)","Population")

defined function dfPopulation

Modifico los datos de entrada para que el formato fecha se adecue al TimeStamp de Spark

In [11]:
def dfCovidClean = dfCovid
    .select($"*",$"dateRep",translate($"dateRep","/","-").as("date"))
    .drop("dateRep")

defined function dfCovidClean

In [12]:
def dfCovidDate = dfCovidClean
    .select($"*",col("date"),to_date(col("date"),"dd-MM-yyyy").as("to_date"))

defined function dfCovidDate

### Media diaria de infecciones por número de habitante

In [13]:
def infectionsPerPopulation = dfCovid.join(dfPopulation, $"country" === $"countriesAndTerritories")
        .select($"country",
                $"dateRep" as "date",
                $"cases",
                $"Population",
                $"cases" / $"Population" as "infection Per Population")
        .groupBy("country")
        .avg("infection Per Population")
        .orderBy(desc("avg(infection Per Population)"))

defined function infectionsPerPopulation

### Porcentaje diario de infectados

In [14]:
def diaryInfectionsDF =
dfCovidDate.join(dfPopulation, $"country" === $"countriesAndTerritories")
        .select($"country",
                $"to_date",
                $"day",
                $"month",
                $"cases",
                $"Population",
                $"cases" / $"Population" as "infection Per Population")
        .orderBy($"to_date".asc)

defined function diaryInfectionsDF

# Visualización de eficiencia

In [15]:
val (x, y) = Seq(
    "DataSet " -> runWithOutput(meanInfectionPerPopulationDS.collect),
    "DataSet diary" -> runWithOutput(diaryInfectionPerPopulationDS.collect),    
    "DataFrame" -> runWithOutput(infectionsPerPopulation.collect),
    "DataFrame diary" -> runWithOutput(diaryInfectionsDF.collect),
).unzip

Bar(x, y).plot()

csv at cmd4.sc:6

1 / 1

csv at cmd4.sc:6

2 / 2

csv at cmd5.sc:6

...

2 / 2

collect at cmd14.sc:5

200 / 200

Took 2788


x: Seq[String] = List(
  "DataSet ",
  "DataSet diary",
  "DataFrame",
  "DataFrame diary"
)
y: Seq[Int] = List(10230, 2886, 3236, 2788)
res14_1: String = "plot-a1560895-edec-498d-a775-c83e03a31963"

In [ ]:
ch.cern.sparkmeasure.StageMetrics(spark).runAndMeasure(meanInfectionPerPopulationDS.collect)

In [ ]:
ch.cern.sparkmeasure.StageMetrics(spark).runAndMeasure(diaryInfectionPerPopulationDS.collect)

In [ ]:
ch.cern.sparkmeasure.StageMetrics(spark).runAndMeasure(
    infectionsPerPopulation.collect
    )

In [ ]:
ch.cern.sparkmeasure.StageMetrics(spark).runAndMeasure(
    diaryInfectionsDF.collect
    )

# Visualización de datos con plotly

## media de infecciones por densidad de población

In [ ]:
def (x,y) = infectionsPerPopulation.collect.map(r=>(r(0).toString, r(1).toString)).toList.unzip
Bar(x, y).plot()

## porcentaje diario de infectados en España

In [ ]:
def (x,y) = diaryInfectionsDF.filter($"country" === "Spain").collect.map(r=>(r(1).toString, r(6).toString)).toList.unzip
Bar(x, y).plot()

## comparacion entre paises de crecimiento de la enfermedad

In [ ]:
val y = diaryInfectionsDF.filter($"country" === "Spain").select($"infection Per Population").
    collect.map(r => r(0).toString.toDouble).toList

val x = diaryInfectionsDF.filter($"country" === "Spain").select($"to_date").collect.toList.map(_.toString)

val y1 = diaryInfectionsDF.filter($"country" === "Italy").select($"infection Per Population").
    collect.map(r => r(0).toString.toDouble).toList
val x1 = diaryInfectionsDF.filter($"country" === "Italy").select($"to_date").collect.toList.map(_.toString)

val data = Seq(
    Scatter(x,y).withName("Spain"),
    Scatter(x1,y1,mode = ScatterMode(ScatterMode.Lines),
  line = Line(color = Color.StringColor("#7F7F7F"))).withName("Italy")
).map(_.withFill(Fill.ToNextY).withStackgroup("A"))

plot(data)